In [3]:
# Importe
from xml.etree import ElementTree as ET
import random
from random import *
import re
import networkx as nx
import matplotlib.pyplot as plt
import os
G=nx.Graph()

# Ein- und Ausgabedateien: Achtung: OdeNet in der Version mit einem Eintrag pro Zeile, wenn man die Datei ändern will!

#de_wn = open(r"C:\Users\melaniesiegel\Documents\05_Projekte\WordNet\OdeNet\odenet.git\trunk\deWordNet.xml","r",encoding="utf-8")
#de_wn = open(r"C:\Users\melaniesiegel\Documents\05_Projekte\WordNet\OdeNet\deWNaccess\odenet_oneline.xml","r",encoding="utf-8")
#en_wn = open(r"C:\Users\melaniesiegel\Documents\05_Projekte\WordNet\English_WN\english-wordnet-2020.xml","r",encoding="utf-8")

de_wn_file = "/content/drive/My Drive/Colab Notebooks/AutomaticOnt/odenet/deWordNet.xml"
de_wn = open(de_wn_file,"r",encoding="utf-8")

en_wn_file = "/content/drive/My Drive/Colab Notebooks/AutomaticOnt/odenet/english_wn_xml/wn_god.xml"
en_wn = open(en_wn_file,"r",encoding="utf-8")

#out_wn = open(r"C:\Users\melaniesiegel\Documents\05_Projekte\WordNet\OdeNet\deWNaccess\odenet_oneline.xml","w",encoding="utf-8")

tree = ET.parse(de_wn)

root = tree.getroot()

lexicon = root.find('Lexicon')

entree = ET.parse(en_wn)

enroot = entree.getroot()

enlexicon = enroot.find('Lexicon')

de_wn.close()
en_wn.close()

## Definitionen, die für die Methoden gebraucht werden

# Alle vorhandenen Informationen zu einem Wort zugreifen

def check_word_lemma(word_to_check):    
    for lexentry in lexicon.iter('LexicalEntry'):
        lemma = lexentry.find('Lemma')
        lemma_value = lemma.attrib['writtenForm']
        lemma_id = lexentry.attrib['id']
        if lemma_value == word_to_check:
            pos = lemma.attrib['partOfSpeech']
            senses = []
            for sense in lexentry.iter('Sense'):
                sense_id = sense.attrib['id']
                synset_id = sense.attrib['synset']
#                senserelation_type = lexentry.find('SenseRelation').attrib['relType']
#                senserelation_target = lexentry.find('SenseRelation').attrib['target']
                senses.append([sense_id,synset_id])
#            print("LEMMA: " + lemma_value + "\nPOS: " + pos + "\nSENSE ID: " + sense_id)
            return(lemma_id, lemma_value, pos, senses)

def words_in_synset(id):
    words = []
    for lexentry in lexicon.iter('LexicalEntry'):
        for sense in lexentry.iter('Sense'):
            if sense.attrib['synset'] == id:
                lemma = lexentry.find('Lemma').attrib['writtenForm']
                words.append(lemma)
    return(words)

def en_words_in_synset(id):
    words = []
    for lexentry in enlexicon.iter('LexicalEntry'):
        for sense in lexentry.iter('Sense'):
            if sense.attrib['synset'] == id:
                lemma = lexentry.find('Lemma').attrib['writtenForm']
                words.append(lemma)
    return(words)

def check_synset(id):
    words = words_in_synset(id)
    for synset in lexicon.iter('Synset'):
        if id == synset.attrib['id']:
            ili = synset.attrib['ili']
            try:
                en_definition = synset.attrib["{http://purl.org/dc/elements/1.1/}description"]
            except KeyError:
                en_definition = []
            if synset.find('Definition') != None:
                de_definition = synset.find('Definition').text.strip()
            else:
                de_definition = []
            relations = []
            for relation in synset.iter('SynsetRelation'):
                reltype = relation.attrib['relType']
                reltarget = relation.attrib['target']
                relations.append((reltype,reltarget))
            return(ili,en_definition,de_definition, relations, words)

def check_word_id(word_id):    
    for lexentry in lexicon.iter('LexicalEntry'):
        lemma = lexentry.find('Lemma')
        lemma_value = lemma.attrib['writtenForm']
        lemma_id = lexentry.attrib['id']
        if lemma_id == word_id:
            pos = lemma.attrib['partOfSpeech']
            senses = []
            for sense in lexentry.iter('Sense'):
                sense_id = sense.attrib['id']
                synset_id = sense.attrib['synset']
                senses.append([sense_id,synset_id])
            #print("LEMMA: " + lemma_value + "\nPOS: " + pos + "\nSENSE ID: " + sense_id)

def find_all_lexentries(word_to_check):
    for lexentry in lexicon.iter('LexicalEntry'):
        lemma = lexentry.find('Lemma')
        lemma_value = lemma.attrib['writtenForm']
        lemma_id = lexentry.attrib['id']
        word_id_list = []
        if lemma_value == word_to_check:
            word_id_list.append(lemma_id)
        for wid in word_id_list:
            check_word_id(wid)            
        
def hypernyms_word(word):
    lemma_id, lemma, pos, senses = check_word_lemma(word)
    hyp_list = []
    for sense in senses:
        (ili,definition,de_definition, relations, words) = check_synset(sense[1])
        for relation in relations:
            if relation[0] == "hypernym":
                hypernym_synset = relation[1]
                hypernym_words = words_in_synset(relation[1])
#            else:
#                hypernym_synset = []
#                hypernym_words = []               
                hyp_list.append((sense[0],hypernym_synset,hypernym_words))
    return(hyp_list)

def hypernyms(sense):
    hyp_list = []
    (ili,definition,de_definition, relations, words) = check_synset(sense)
    for relation in relations:
        if relation[0] == "hypernym":
            hypernym_synset = relation[1]
            hypernym_words = words_in_synset(relation[1])
            hyp_list.append((hypernym_synset,hypernym_words))
    return(hyp_list)

def hyponyms(sense):
    hyp_list = []
    (ili,definition,de_definition, relations, words) = check_synset(sense)
    for relation in relations:
        if relation[0] == "hyponym":
            hyponym_synset = relation[1]
            hyponym_words = words_in_synset(relation[1])
            hyp_list.append((hyponym_synset,hyponym_words))
    return(hyp_list)

def hyponyms_word(word):
    lemma_id, lemma, pos, senses = check_word_lemma(word)
    hyp_list = []
    for sense in senses:
        (ili,definition,de_definition, relations, words) = check_synset(sense[1])
        for relation in relations:
            if relation[0] == "hyponym":
                hyponym_synset = relation[1]
                hyponym_words = words_in_synset(relation[1])
 #           else:
 #               hyponym_synset = []
 #               hyponym_words = []               
                hyp_list.append((sense[0],hyponym_synset,hyponym_words))
    return(hyp_list)

def meronyms_word(word):
    lemma_id, lemma, pos, senses = check_word_lemma(word)
    mero_list = []
    for sense in senses:
        (ili,definition,de_definition, relations, words) = check_synset(sense[1])
        for relation in relations:
            if relation[0] == "mero_part":
                meronym_synset = relation[1]
                meronym_words = words_in_synset(relation[1])
                mero_list.append((sense[0],meronym_synset,meronym_words))
    return(mero_list)

def holonyms_word(word):
    lemma_id, lemma, pos, senses = check_word_lemma(word)
    holo_list = []
    for sense in senses:
        (ili,definition,de_definition, relations, words) = check_synset(sense[1])
        for relation in relations:
            if relation[0] == "holo_part" or relation[0] == "holo_member":
                holo_synset = relation[1]
                holo_words = words_in_synset(relation[1])
                holo_list.append((sense[0],holo_synset,holo_words))
    return(holo_list)

def antonyms_word(word):
    lemma_id, lemma, pos, senses = check_word_lemma(word)
    anto_list = []
    for sense in senses:
        (ili,definition,de_definition, relations, words) = check_synset(sense[1])
        for relation in relations:
            if relation[0] == "antonym":
                antonym_synset = relation[1]
                antonym_words = words_in_synset(relation[1])
                anto_list.append((sense[0],antonym_synset,antonym_words))
    return(anto_list)

def words2ids(wordlist):
    word_id_list = []
    for word in wordlist:
        try:
            lemma_id, lemma, pos, senses = check_word_lemma(word)
            word_id_list.append(lemma_id)
        except:
            print(word + " NOT IN ODENET")
    return(word_id_list)
    


# Visualisierungen

seen = set()

def recurse_hyper(s,word):
    if not s in seen:
        seen.add(s)
        G.add_node(word)
        hypers = hypernyms(s)
        print(str(hypers))
#        if len(hypers) > 0:
        for h in hypers:
            G.add_node(h[1][0])
            G.add_edge(word,h[1][0])
            recurse_hyper(h[0],h[1][0])

def recurse_hypo(s,word):
    if not s in seen:
        seen.add(s)
        G.add_node(word)
        hypos = hyponyms(s)
        for h in hypos:
            G.add_node(h[1][0])
            G.add_edge(word,h[1][0])
            recurse_hypo(h[0],h[1][0])

def recurse(s,word):
    if not s in seen:
        seen.add(s)
        G.add_node(word)
        hypers = hypernyms(s)
        hypos = hyponyms(s)
        print(str(hypers))
        print(str(hypos))
#        if len(hypers) > 0:
        for h in hypers:
            G.add_node(h[1][0])
            G.add_edge(word,h[1][0])
            recurse_hyper(h[0],h[1][0])
        for h in hypos:
            G.add_node(h[1][0])
            G.add_edge(word,h[1][0])
            recurse_hypo(h[0],h[1][0])

#recurse_hyper(sense,word)

#recurse_hypo(sense,word)

def visualize_hypernyms():
    word = input("Welches Wort? ")
    lemma, pos, senses = check_word_lemma(word)
    print("Ich habe diese Senses dafür: ")
    for s in senses:
        print("SYNSET: " + s[1])
        print(check_synset(s[1]))
    sense = input("Welcher Sense? ")      
    recurse_hyper(sense,word)
    print (G.nodes(data=True))
    nx.draw_networkx(G, width=2, with_labels=True)
    plt.show()

def visualize_hyponyms():
    word = input("Welches Wort? ")
    lemma, pos, senses = check_word_lemma(word)
    print("Ich habe diese Senses dafür: ")
    for s in senses:
        print("SYNSET: " + s[1])
        print(check_synset(s[1]))
    sense = input("Welcher Sense? ")
    recurse_hypo(sense,word)
    print (G.nodes(data=True))
    nx.draw_networkx(G, width=2, with_labels=True)
    plt.show()


               
## Die Klasse für den Zugriff auf OdeNet

class OdeNet(object):
    def word_info(object):
        if check_word_lemma(object):
          (lemma_id, lemma_value, pos, senses) = check_word_lemma(object)
          # print (lemma_value + " " + pos + " ")
          # for sense in senses:
          #     print("SENSE: " + str(sense[1]) + "  " + str(check_synset(sense[1])) + "\n")
          # print("HYPERNYMS: " + str(hypernyms_word(object)))
          # print("HYPONYMS: " + str(hyponyms_word(object)))
          # print("MERONYMS: " + str(meronyms_word(object)))
          # print("HOLONYMS: " + str(holonyms_word(object)))
          # print("ANTONYMS: " + str(antonyms_word(object)))
          find_all_lexentries(object) 
          return [senses, hypernyms_word(object), hyponyms_word(object)]
        else:
          return ""                   
    pass
    def check_ili_in_pwn(object):
        for lexentry in enlexicon.iter('LexicalEntry'):
            lemma = lexentry.find('Lemma')
            lemma_value = lemma.attrib['writtenForm']
            if lemma_value == object:
                pos = lemma.attrib['partOfSpeech']
                senses = []
                for sense in lexentry.iter('Sense'):
                    synset_id = sense.attrib['synset']
                    senses.append(synset_id)
                for sense in senses:
                    words = en_words_in_synset(sense)
                    print(str(words))
                    for synset in enlexicon.iter('Synset'):
                        if sense == synset.attrib['id']:
                            ili = synset.attrib['ili']
                            print(str(ili))
                            definition = synset.find('Definition').text.strip()
                            print(str(definition))
    pass
    def word_id(object):
        lemma_id, lemma_value, pos, senses = check_word_lemma(object)
        return lemma_id
    pass
    def visualize(object):
        lemma_id, lemma_value, pos, senses = check_word_lemma(object)
        if len(senses) == 1:
            sense = senses[0][1]
        else:
            print("Ich habe diese Senses dafür: ")
            for s in senses:
                print("SYNSET: " + s[1])
                print(check_synset(s[1]))
            sense = input("Welcher Sense? ")      
        recurse(sense,object)
        print (G.nodes(data=True))
        nx.draw_networkx(G, width=2, with_labels=True)
        plt.show()
    pass



In [4]:
# Importe


## Funktionen für die Arbeit mit OdeNet

# Informationen zu OdeNet hinzufügen
# nur in oneline-Datei!

# Relationen dem Synset hinzufügen
# add_rel_to_ss(synset,relation,r"C:\Users\melaniesiegel\Documents\05_Projekte\WordNet\OdeNet\deWNaccess\odenet_oneline.xml")


def add_rel_to_ss(synset,relation,wordnetfile):
    if synset not in relation:
        de_wn = open(wordnetfile,"r",encoding="utf-8")
        lines = de_wn.readlines()
        de_wn.close() 
        out_odenet = open(wordnetfile,"w",encoding="utf-8")
        ss_string = '<Synset id="' + synset + '"'
        for line in lines:
            if ss_string in line and relation not in line:
                if '<Example>' in line:
                    line = line.replace('<Example>',relation + '<Example>')
                elif '</Synset>' in line:
                    line = line.replace('</Synset>',relation + '</Synset>')
                else:
                    line = line.replace('/>', '>' + relation + '</Synset>')
                print(line)
            out_odenet.write(line)
        out_odenet.close()
        
# Attribute in Synsets verändern, z.B. ili
# change_attribute_in_ss('odenet-412-a','ili','i10007',r"C:\Users\melaniesiegel\Documents\05_Projekte\WordNet\OdeNet\deWNaccess\odenet_oneline.xml")

def change_attribute_in_ss(synset,att,value,wordnetfile):
        de_wn = open(wordnetfile,"r",encoding="utf-8")
        lines = de_wn.readlines()
        de_wn.close()
        out_odenet = open(wordnetfile,"w",encoding="utf-8")
        ss_string = '<Synset id="' + synset + '"'
        for line in lines:
            if ss_string in line:
                line = re.sub(att + '="[a-zA-Z0-9]*"', att + '="'+ value +'"', line)
                print(line)
            out_odenet.write(line)
        out_odenet.close()

# Attribute zu Lexentries hinzufügen, z.B. confidenceScore

def change_attribute_in_lexentry(lemma,att,value,wordnetfile):
        de_wn = open(wordnetfile,"r",encoding="utf-8")
        lines = de_wn.readlines()
        de_wn.close()
        out_odenet = open(wordnetfile,"w",encoding="utf-8")
        lemma_string = 'Lemma writtenForm="' + lemma + '"'
        for line in lines:
            if lemma_string in line and att not in line:
                line = line.replace("><Lemma writtenForm=",' ' + att + '="'+ value + '"' + "><Lemma writtenForm=")
                print(line)
            out_odenet.write(line)
        out_odenet.close()
    
# Attribute zu Senses hinzufügen, z.B. note
# Sense id="w18234_4118-n" synset="odenet-4118-n" note="PHON:boːt"/>

def add_attribute_to_sense(sense_id,synset,att,value,wordnetfile):
        de_wn = open(wordnetfile,"r",encoding="utf-8")
        lines = de_wn.readlines()
        de_wn.close()
        out_odenet = open(wordnetfile,"w",encoding="utf-8")
        sense_string = 'Sense id="' + sense_id + '" synset="'+ synset + '"'
        for line in lines:
            if sense_string in line and value not in line:
#            if sense_string in line:
                line = line.replace(sense_string,sense_string + ' ' + att + '="'+ value + '"')
                print(line)
            out_odenet.write(line)
        out_odenet.close()
        

    
# Definitionen zu einem Synset hinzufügen    
def add_definition_to_ss(synset,definition, wordnetfile):
        de_wn = open(wordnetfile,"r",encoding="utf-8")
        lines = de_wn.readlines()
        de_wn.close()
        out_odenet = open(wordnetfile,"w",encoding="utf-8")
        ss_string = '<Synset id="' + synset + '"'
        definition_string = "<Definition>" + definition + "</Definition>"
        for line in lines:
            if ss_string in line and "<Definition>" not in line:
                if '<Example>' in line:
                    line = line.replace('<Example>', definition_string + '<Example>')
                elif '<SynsetRelation' in line:
                    line = line.replace('<SynsetRelation', definition_string + '<SynsetRelation',1)
                elif '</Synset>' in line:
                    line = line.replace('</Synset>', definition_string + '</Synset>')
                else:
                    line = line.replace('/>', '>' + definition_string + '</Synset>')
                print(line)
            out_odenet.write(line)
        out_odenet.close()

# Die englische Definition löschen
def delete_english_definition(synset, wordnetfile):
        de_wn = open(wordnetfile,"r",encoding="utf-8")
        lines = de_wn.readlines()
        de_wn.close()
        out_odenet = open(wordnetfile,"w",encoding="utf-8")
        ss_string = '<Synset id="' + synset + '"'
        for line in lines:
            if ss_string in line and "dc:description" in line:
                line=re.sub('dc:description=".*"','',line)
                print(line)
            out_odenet.write(line)
        out_odenet.close()

# Ein Beispiel einfügen

def add_example_to_ss(synset,example, wordnetfile):
        de_wn = open(wordnetfile,"r",encoding="utf-8")
        lines = de_wn.readlines()
        de_wn.close()
        out_odenet = open(wordnetfile,"w",encoding="utf-8")
        ss_string = '<Synset id="' + synset + '"'
        example_string = "<Example>" + example + "</Example>"
        for line in lines:
            if ss_string in line and "<Example>" not in line:
                if '</Synset>' in line:
                    line = line.replace('</Synset>', example_string + '</Synset>')
                print(line)
            out_odenet.write(line)
        out_odenet.close()

# Die Version ohne Zeilenumbruch als Pretty Print speichern

def prettyprint_odenet():
    oneline_odenet = open(r"C:\Users\melaniesiegel\Documents\05_Projekte\WordNet\OdeNet\deWNaccess\odenet_oneline.xml","r", encoding="utf-8")
    lines = oneline_odenet.readlines()
    oneline_odenet.close()
    pretty_odenet = open(r"C:\Users\melaniesiegel\Documents\05_Projekte\WordNet\OdeNet\odenet.git\trunk\deWordNet.xml","w",encoding="utf-8")
    for line in lines:
        line = line.replace("<Lemma","\n\t<Lemma")
        line = line.replace("<Sense","\n\t<Sense")
        line = line.replace("</Sense","\n\t</Sense")
        line = line.replace("</LexicalEntry>","\n</LexicalEntry>")
        line = line.replace("<SynsetRelation","\n\t<SynsetRelation")
        line = line.replace("<Definition>","\n\t<Definition>")
        line = line.replace("<Example>","\n\t<Example>")
        line = line.replace("</Synset>","\n</Synset>")
        line = line.replace("<SyntacticBehaviour","\n\t<SyntacticBehaviour")
        pretty_odenet.write(line)
    pretty_odenet.close()


In [5]:
#OdeNet.word_info("Auseinandersetzung")
#OdeNet.word_info("Verp")
# senses, hyper, hypo

In [6]:
#OdeNet.check_ili_in_pwn("show")

In [7]:
# Hyponyme und Hyperonyme visualisieren
#OdeNet.visualize("Auseinandersetzung")
#OdeNet.visualize("Fall")

#Text Lemmatization using Tiger Corpus

Source: https://github.com/WZBSocialScienceCenter/germalemma

In [8]:
pip install -U germalemma

     |████████████████████████████████| 2.3MB 3.9MB/s 
     |████████████████████████████████| 22.1MB 62.6MB/s 
     |████████████████████████████████| 3.0MB 48.5MB/s 


In [9]:
from germalemma import GermaLemma

lemmatizer = GermaLemma()

In [10]:
# passing the word and the POS tag ("N" for noun)
lemma = lemmatizer.find_lemma('Feinstaubbelastungen', 'N')
print(lemma)

Feinstaubbelastung


#Get POS of the words

We are using Spacy

In [11]:
pip install --upgrade spacy

     |████████████████████████████████| 10.0MB 4.3MB/s 
     |████████████████████████████████| 2.1MB 47.5MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [12]:
!python -m spacy download de_core_news_md

     |████████████████████████████████| 47.0MB 1.2MB/s 
  Created wheel for de-core-news-md: filename=de_core_news_md-2.3.0-cp36-none-any.whl size=47054219 sha256=9c12ffb23d2f9c444a51cec01fb31925de7478a09e9ffd126d629bb3fd1ed623
  Stored in directory: /tmp/pip-ephem-wheel-cache-hta0_8r2/wheels/61/d0/4e/98754e2456d0650faecf781724db11f925c1610653914e6ee4
Successfully built de-core-news-md
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_md')


In [13]:
import spacy
import de_core_news_md

In [14]:
nlp = de_core_news_md.load()

In [27]:
def getPos(word):
  text = (word) 
  doc = nlp(text)
  for token in doc: 
    if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "X":
      return 'N'
    elif token.pos_ == "VERB":
      return 'V'
    elif token.pos_ == "ADJ" or token.pos_ == "ADV":
      return token.pos_
    else:
      return 'N'

In [16]:
def getLemma(word):
  word_pos = getPos("" + word + "")
  lemma = lemmatizer.find_lemma(word, word_pos)
  return [word, lemma, word_pos]

In [17]:
print(getLemma("Forstwirten"))

['Forstwirten', 'Forstwirten', 'N']


#Process Words

In [18]:
manual_words_list = ['Buchführung',  'Rechnungslegung',  'Buchführung',  'Verwaltungsanweisung',  'Freibetrags',  'Rechtsbehelfsverfahrens ',  'Anwendung',  'Veranlagungszeitraum',  'Ausgangswertgrenze',  'Bayern',  'Buchwert',  'Betriebsvermögen',  'Vermögen ',  'Fall',  'Bescheinigung',  'Oberfinanzdirektion',  'Kirchensteuer',  'Rundverfügung',  'Köln',  'Handelsrecht',  'Gemeind',  'Ausgleichsanspruch ',  'Zustaendigkeit',  'Beschwerd',  'Lohnunternehm',  'Gesellschaftsrecht',  'Ausschlussfrist',  'Ausschlußfrist',  'Schulden',  'Unterlage',  'Düsseldorf',  'Arbeitsvertrag',  'Erklärung',  'Betriebsinhaber',  'Land- und Forstwirte',  'Bundesfinanzhof',  'Frankfurt am Main',  'Förderungsgesetze',  'Allgemeines ',  'Lohnsteuer',  'Einkommensteuerveranlagung',  'Versicherung',  'Zinseinnahmen',  'Schuldzinsen',  'Einführungsschreiben',  'Urteil',  'Zuständigkeit ',  'Kiel',  'Kiel',  'Koblenz',  'Koblenz',  'Gesetz',  'Pachtung',  'Verpachten',  'Pachterlöse',  'Verpachtungserlaß',  'Vermächtnisnehmer',  'Verpächter',  'Viehdurchschnittswerte',  'Bewirtschafter',  'Wirtschaftsjahrs',  'Wirtschaftsjahre',  'Mitteilung',  'Münster',  'Reingewinn',  'Nordrhein-Westfalen',  'Nürnberg',  'Nürnberg',  'Eröffnungsbilanz',  'Betriebsausgaben',  'Betriebseinnahmen',  'Betriebsaufgabe',  'Absatz',  'Teilwerts',  'Teilwert',  'Zahlung',  'Geltungsdauer',  'Privatvermögen',  'Gewinngrenze ',  'Nachweis',  'Eigentum',  'Vermögensgrenze ',  'Bestimmungen',  'Bezugsverfügung',  'Aufzeichnungen ',  'Tilgung',  'Vorschrift',  'Aufforderung',  'Einnahmen',  'Wohnrecht',  'Saarbrücken',  'Saarbrücken',  'Saarland',  'Umsatz',  'Umsätze',  'Handelsvertreters',  'Abfindungen',  'Gesellschaftsvertrages',  'Alleinerben',  'Ehegatt',  'Ehegatte',  'Ausgangsbetrag',  'Erbfolge',  'Zuschlag ',  'Übernahme ',  'Steuerbescheid',  'Abgabenordnung',  'Steuerhinterziehung',  'Steuergesetz',  'Finanzamts ',  'Steuervergünstigung',  'Steuerbegünstigung',  'Besteuerung ',  'Steuerpflichtiger',  'Steuerpflichtigen',  'Pächter', 'Nutzungswert',  'Entnahmewertes',  'Arbeitskraft']

In [19]:
algo_10_files_word_list = ['Abfindung', 'Abs', 'Abschnitt', 'AfA', 'Allgemeines', 'Anbau',
       'Anhalt', 'Anlage', 'Antrags', 'Arbeitsleistung', 'Berechnung',
       'Betracht', 'Betrieb', 'Betriebs', 'Betriebsausgaben',
       'Betriebsinhaber', 'Betriebsvermögen', 'Bewirtschaftung', 'Bodens',
       'Buchführung', 'Buchführungspflicht', 'Buchstabe', 'EStG',
       'Ehegatten', 'Einkünfte', 'Einkünften', 'Erben', 'Erfassung',
       'Erhöhte', 'Erlasses', 'Ermittlung', 'Festsetzung', 'Feststellung',
       'Feststellungen', 'Finanzamt', 'Fläche', 'Flächen',
       'Forstwirtschaft', 'Fragebogen', 'Freibetrag', 'Freibetrags',
       'Gegenüberstellung', 'Gewinn', 'Gewinne', 'Gewinns', 'Gewährung',
       'Grundstück', 'Grundstücke', 'Grundstücken', 'Herstellungskosten',
       'III', 'Inventar', 'Land-', 'Landwirt', 'Nachweis', 'Nutzungen',
       'Obstbau', 'Privatvermögen', 'Schuldzinsen', 'Schätzung',
       'Sonderabschreibungen', 'Sonderausgaben', 'Steuerpflichtigen',
       'Tieren', 'Tierhaltung', 'Tilgung', 'Veranlagung', 'Veräußerung',
       'Veräußerungen', 'Wahlrecht', 'Weinbau', 'Werts',
       'Wirtschaftsgüter', 'Wirtschaftsgütern', 'Wirtschaftsjahr',
       'Wohnung', 'Wurde', 'Ziffer', 'Zuschlag', 'Zuschläge', 'abgesetzt',
       'abweicht', 'angesetzten', 'anzusetzen', 'anzuwenden',
       'aufgeführten', 'ausgewiesenen', 'bekanntgegeben', 'beschäftigten',
       'bewirtschaftet', 'bezeichneten', 'dauernde', 'enthaltenen',
       'entnommen', 'erfaßt', 'erhöhte', 'erhöhten', 'ermitteln',
       'gehörende', 'gehörenden', 'gem', 'genutzte', 'gärtnerische',
       'körperliche', 'land-', 'landwirtschaftlich',
       'landwirtschaftliche', 'landwirtschaftlichen',
       'landwirtschaftlicher', 'maßgebend', 'maßgebende', 'maßgebenden',
       'nachgewiesenen', 'sachlichen', 'sinngemäß', 'unentgeltlich',
       'veräußert', 'vgl', 'zuzurechnen', 'Überlassung', 'überführen']

In [20]:
tag_me_10_files_word_list = ['Abschreibung', 'Absetzung für Abnutzung', 'Altenteil', 'Anlagevermögen', 'Anschaffungskosten', 'Arbeitsrecht', 'Arnsberg', 'Aßberg', 'Ausschlussfrist', 'Außenprüfung', 'Ayten Erten', 'Baumartengruppe', 'Before Present', 'Berufsgenossenschaft', 'Betriebsausgabe', 'Betriebsfinanzamt', 'Betriebsveräußerung', 'Betriebsvermögen', 'Betriebsvermögensvergleich', 'Bodenschätzung', 'Bohne', 'Branntwein', 'Buchführung', 'Bundesautobahn', 'Bundesautobahn ', 'Bundesautobahn 6', 'Bundesfinanzhof', 'Bundesministerium der Finanzen', 'Darfur', 'Dauernde Last', 'Deich', 'Echter Sellerie', 'Einheitswert', 'Einkommensteuer', 'Einkommensteuer (Deutschland)', 'Einkommensteuer-Durchführungsverordnung', 'Einkommensteuer-Richtlinien', 'Einkommensteuererklärung', 'Einkommensteuergesetz', 'Einkommensteuergesetzbuch', 'Einkünfte aus Gewerbebetrieb (Deutschland)', 'Einkünfte aus nichtselbständiger Arbeit (Deutschland)', 'Emanuele Vona', 'Emrah Eren', 'Endivie', 'Enkirch', 'Erbbaurecht', 'Erbbauzins', 'Erbschaftsteuer', 'Erbse', 'Erf (Fluss)', 'Errungenschaftsgemeinschaft', 'Erté', 'Ertrag', 'Esparsetten', 'Ester', 'Fahrnisgemeinschaft', 'Feldsalat', 'Finanzamt', 'Flurbereinigung', 'Forstwirtschaft', 'Fortgesetzte Gütergemeinschaft', 'Freinsheim', 'Friedelsheim', 'Fruchtfolge', 'Gabelstapler', 'Gebäudeversicherung', 'Gemeiner Wert', 'Gemüse', 'Gemüsebau', 'Gemüsespargel', 'Gerste', 'Geschäftsjahr', 'Gesetzliche Erbfolge', 'Getreide', 'Gewinn', 'Gönnheim', 'Grafschaft Saarbrücken', 'Grundlagenbescheid', 'Grundsteuer', 'Grundsteuer (Deutschland)', 'Grünland', 'Gurke', 'Gütergemeinschaft (Ehe)', 'Güterrecht', 'Hackfrucht', 'Hafer', 'Handelsgesetzbuch', 'Handelsregister', 'Haushaltsjahr', 'Hefen', 'Herstellungskosten', 'Hofladen', 'Hopfen', 'Hypothekengewinnabgabe', 'Imker', 'Immaterieller Vermögensgegenstand', 'Infimum und Supremum', 'Kalamität', 'Kallstadt', 'Karotte', 'Kataster', 'Kaufmann', 'Kinderfreibetrag', 'Kleegras', 'Kleingarten', 'Kleingewerbe', 'Koblenz', 'Kohl', 'Kohlrabi', 'Kopfkohl', 'Kopfsalat', 'Korken', 'Kostenmiete', 'Kreis Brilon', 'Kreis Olpe', 'Land- und forstwirtschaftlicher Betrieb', 'Landwirtschaftliche Alterskasse', 'Lauch', 'Laufendes Gut', 'Leistadt', 'Lohnsteuer', 'Lohnsteuer (Deutschland)', 'Lohnsteueraußenprüfung', 'Lohnunternehmer', 'Lüdenscheid', 'Lupinen', 'Luzerne', 'Maischen', 'Meschede', 'Mietspiegel', 'Minderung der Erwerbsfähigkeit', 'Mitunternehmerschaft', 'Mosel (Weinbaugebiet)', 'Naci Erdem', 'Nießbrauch', 'Nürnberg', 'Nutzungswert', 'Oberfinanzdirektion', 'Oberfinanzdirektion Münster', 'Obstbau', 'Orhan Eren', 'Pilzanbau', 'Reblaus', 'Rettiche', 'Roggen', 'Rote Bete', 'Rotkohl', 'Rudolf Felzmann (Heimatforscher)', 'Ruwer', 'Saatzucht', 'Shinyanga (Region)', 'Sonderabschreibung', 'Sonderkultur', 'Sondernutzung', 'Sozialbrache', 'Speyer', 'Spinat', 'ST 16', 'Ständigkeit', 'Stattgabe', 'Steuerbescheid', 'Steuererklärung', 'Steuerermäßigung', 'Steuergefährdung', 'Steuerhinterziehung (Deutschland)', 'Steuersubvention', 'Stille Reserven', 'Teichwirtschaft', 'Teilwert', 'Tomate', 'Trester (Pressrückstände)', 'Umsatzsteuer', 'Umsatzsteuergesetz', 'Ungstein', 'Veranlagung (Steuerrecht)', 'Veranlagungszeitraum', 'Veräußerungsgewinn', 'Verfahrensrecht', 'Vergleichsmiete', 'Vermögensteuer', 'Vertova', 'Verwaltungsrecht (Deutschland)', 'Vorbehalt der Nachprüfung', 'Vorsteuerpauschale', 'Weinbau', 'Weinherstellung', 'Weißkohl', 'Weizen', 'Wicken (Vicia)', 'Wirsing', 'Wirtschaftliche Betrachtungsweise', 'Wirtschaftsgut (Steuerlehre)', 'Zierpflanzenbau', 'Zwiebel']

**Process Manual words**

In [21]:
import pandas as pd

**Clean Tag-me words**

In [22]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
german_stop_words = stopwords.words('german')

In [24]:
tag_me_word_list = []
for word in tag_me_10_files_word_list:
  if(len(word.split(' '))) == 1:
    tag_me_word_list.append(word)
  else:
    word_arr = word.split(' ')
    for item in word_arr:
      if item not in german_stop_words and item != '' and not item.isnumeric() and not item.startswith('('):
        tag_me_word_list.append(item)

In [25]:
def lemmatize_pos_wordlist(wordList):
  temp_list = []
  for word in wordList:
    temp_l = getLemma(word)
    temp_list.append(temp_l)
  words_df = pd.DataFrame(temp_list, columns=['Raw Word', 'Lemma_word', 'POS'])
  return words_df

In [29]:
tagme_words_df = lemmatize_pos_wordlist(tag_me_word_list)
algo_words_df = lemmatize_pos_wordlist(algo_10_files_word_list)
manual_words_df = lemmatize_pos_wordlist(manual_words_list)
algo_words_df.head()

,Raw Word,Lemma_word,POS
0,Abfindung,Abfindung,N
1,Abs,Abs,N
2,Abschnitt,Abschnitt,N
3,AfA,AfA,N
4,Allgemeines,Allgemeines,N


**Store Hyper and Hyponyms**

For Manual List

In [30]:
def get_hyper_list(words_df):
  result_hyper_list = []
  for index, row in words_df.iterrows():
    result = OdeNet.word_info(row["Lemma_word"])
    if result != '':
      hyper_obj = result[1]
      for item in hyper_obj:
        temp_list = list(item)
        sense = temp_list[0]
        hypers = temp_list[2]
        result_hyper_list.append([row["Raw Word"], row["Lemma_word"], row["POS"], sense, hypers])
    else:
      result_hyper_list.append([row["Raw Word"], row["Lemma_word"], row["POS"], '', ''])

  df = pd.DataFrame(result_hyper_list, columns=["Raw_word", "Lemma_word", "POS", "Sense", "Hypernyms"])
  return df

In [31]:
def get_hypo_list(words_df):
  result_hypo_list = []
  for index, row in words_df.iterrows():
    result = OdeNet.word_info(row["Lemma_word"])
    if result != '':
      hypo_obj = result[2]
      for item in hypo_obj:
        temp_list = list(item)
        sense = temp_list[0]
        hypos = list(temp_list[2])
        result_hypo_list.append([row["Raw Word"], row["Lemma_word"], row["POS"], sense, hypos])
    else:
      result_hypo_list.append([row["Raw Word"], row["Lemma_word"], row["POS"], '', ''])

  df = pd.DataFrame(result_hypo_list, columns=["Raw_word", "Lemma_word", "POS", "Sense", "Hyponyms"])
  return df

In [32]:
manual_hypo_df = get_hypo_list(manual_words_df)
algo_hypo_df = get_hypo_list(algo_words_df)
tagme_hypo_df = get_hypo_list(tagme_words_df)

In [33]:
manual_hyper_df = get_hyper_list(manual_words_df)
algo_hyper_df = get_hyper_list(algo_words_df)
tagme_hyper_df = get_hyper_list(tagme_words_df)

**Saving results to Excel**

In [34]:
import pandas as pd

In [35]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
#writer = pd.ExcelWriter('/content/drive/My Drive/Colab Notebooks/AutomaticOnt/odenet/hyper_hypo_all.xlsx', engine='xlsxwriter')
with pd.ExcelWriter('/content/drive/My Drive/Colab Notebooks/AutomaticOnt/odenet/hyper_hypo_all.xlsx') as writer:
  # Write each dataframe to a different worksheet.
  manual_hyper_df.to_excel(writer, sheet_name='Manual - Hypernyms')
  manual_hypo_df.to_excel(writer, sheet_name='Manual - Hyponyms')

  algo_hyper_df.to_excel(writer, sheet_name='Algo - Hyperyms')
  algo_hypo_df.to_excel(writer, sheet_name='Algo - Hyponyms')

  tagme_hyper_df.to_excel(writer, sheet_name='Tagme - Hypernyms')
  tagme_hypo_df.to_excel(writer, sheet_name='Tagme - Hyponyms')

In [36]:
hyper_df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/AutomaticOnt/odenet/hyper_hypo_all.xlsx', sheet_name='Manual - Hypernyms')
hypo_df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/AutomaticOnt/odenet/hyper_hypo_all.xlsx', sheet_name='Manual - Hyponyms')

In [ ]:
hypo_df.head(10)

,Unnamed: 0,Raw_word,Lemma_word,POS,Sense,Hyponyms
0,0,Rechnungslegung,Rechnungslegung,N,NaN,NaN
1,1,Verwaltungsanweisung,Verwaltungsanweisung,N,NaN,NaN
2,2,Freibetrags,freibetrags,ADV,NaN,NaN
3,3,Rechtsbehelfsverfahrens,Rechtsbehelfsverfahrens,N,NaN,NaN
4,4,Anwendung,Anwendung,N,w8296_1685-n,"['Regenerierung', 'Wiedergewinnung', 'Wiederve..."
5,5,Anwendung,Anwendung,N,w8296_1685-n,"['Eigennutz', 'zum privaten Nutzen', 'zum pers..."
6,6,Anwendung,Anwendung,N,w8296_1685-n,"['Hausgebrauch', 'Heimgebrauch']"
7,7,Anwendung,Anwendung,N,w8296_1685-n,"['Verwendung', 'Auswertung', 'Nutzbarmachung',..."
8,8,Anwendung,Anwendung,N,w8296_5980-n,"['Regenerierung', 'Wiedergewinnung', 'Wiederve..."
9,9,Anwendung,Anwendung,N,w8296_5980-n,"['Anwendung', 'Programm', 'Applikation', 'Soft..."


In [ ]:
hyper_df.head(10)

,Unnamed: 0,Raw_word,Lemma_word,POS,Sense,Hypernyms
0,0,Rechnungslegung,Rechnungslegung,N,NaN,NaN
1,1,Verwaltungsanweisung,Verwaltungsanweisung,N,NaN,NaN
2,2,Freibetrags,freibetrags,ADV,NaN,NaN
3,3,Rechtsbehelfsverfahrens,Rechtsbehelfsverfahrens,N,NaN,NaN
4,4,Anwendung,Anwendung,N,w8296_1685-n,"['Versuch', 'Vorsatz', 'Unternehmung']"
5,5,Anwendung,Anwendung,N,w8296_5980-n,"['Versuch', 'Vorsatz', 'Unternehmung']"
6,6,Anwendung,Anwendung,N,w8296_12037-n,"['Anordnung', 'Vorschrift', 'Anweisung', 'Instruktion']"
7,7,Anwendung,Anwendung,N,w8296_12037-n,"['Verwendung', 'Anwendung', 'Einsatz', 'Applikation']"
8,8,Veranlagungszeitraum,Veranlagungszeitraum,N,NaN,NaN
9,9,Ausgangswertgrenze,Ausgangswertgrenze,N,NaN,NaN
